In [ ]:
import pprint
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import tifffile
from careamics_portfolio import PortfolioManager
from matplotlib.pyplot import imshow

from careamics.engine import Engine

### Import Dataset Portfolio

In [ ]:
# Explore portfolio
portfolio = PortfolioManager()
print(portfolio.denoising)

In [ ]:
# Download files
root_path = Path("./data")
files = portfolio.denoising.N2V_SEM.download(root_path)
print(f"List of downloaded files: {files}")

### Visualize training data

In [ ]:
# Load images
train_image = tifffile.imread(files[0])
print(f"Train image shape: {train_image.shape}")
imshow(train_image, cmap="gray")

### Visualize validation data

In [ ]:
val_image = tifffile.imread(files[1])
print(f"Validation image shape: {val_image.shape}")
imshow(val_image, cmap="gray")

In [ ]:
data_path = Path(root_path / "n2v_sem")
train_path = data_path / "train"
val_path = data_path / "val"

train_path.mkdir(parents=True, exist_ok=True)
val_path.mkdir(parents=True, exist_ok=True)

shutil.copy(root_path / files[0], train_path / "train_image.tif")
shutil.copy(root_path / files[1], val_path / "val_image.tif")

### Initialize the Engine

Engine contains the dataloading pipeline and the model training logic. We'll initialize the engine with the config file, but it can also be initialized from a pre-trained checkpoint.

Please take as look at the [documentation](https://careamics.github.io) to see the full list of parameters and configuration options

In [ ]:
engine = Engine(config_path="n2v_2D_SEM.yml")

### Visualize training configuration

In [ ]:
pprint.PrettyPrinter(indent=2).pprint(engine.cfg.model_dump(exclude_optionals=False))

### Run training 

We need to specify the paths to training and validation data

In [ ]:
train_stats, val_stats = engine.train(train_path=train_path, val_path=val_path)

### Visualize statistics

In [ ]:
plt.plot([next(iter(d.values())) for d in train_stats], label="Train loss")
plt.plot([next(iter(d.values())) for d in val_stats], label="Validation loss")
plt.legend(loc="best")
plt.xlabel("Epoch")

### Run prediction

We need to specify the path to the data we want to denoise

In [ ]:
preds = engine.predict(input=train_path, tile_shape=[256, 256], overlaps=[48, 48])

### Visualize the prediction

In [ ]:
imshow(preds.squeeze(), cmap="gray")

### Export to bioimage.io

In [ ]:
engine.save_as_bioimage(engine.cfg.experiment_name + "bioimage.zip")